In [ ]:
!pip install mediapipe

In [ ]:
!pip install opencv-python

# MediaPipe

MediaPipe offers open source cross-platform, customizable ML solutions for live and streaming media. 
End-to-End acceleration: Built-in fast ML inference and processing accelerated even on common hardware. 
Build once, deploy anywhere: Unified solution works across Android, iOS, desktop/cloud, web and IoT.

# face mesh 3d with body pose on images

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in glob.glob("./images/testing/*.jpg")}

In [2]:
for name, image in images.items():
    image =cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#    plt.imshow(image)
#    plt.show()


In [3]:
import mediapipe as mp
mp_holistic = mp.solutions.holistic

#help(mp_holistic.Holistic)

In [4]:
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [5]:
# Initialize MediaPipe Pose.
import cv2
import time
import mediapipe as mp


class PoseDetector:

    def __init__(self, mode = False, upBody = False, smooth=True, detectionCon = 0.5, trackCon = 0.5):

        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth, self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)

        return img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS

    def getPosition(self, img, draw=True):
        lmList= []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return lmList




pTime = 0
cap = cv2.VideoCapture(0)

detector = PoseDetector()















i=0
img_array = []

for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Pose.


    img, p_landmarks, p_connections = detector.findPose(image, False)
    
    
    # draw points
    mp.solutions.drawing_utils.draw_landmarks(img, p_landmarks, p_connections)
    lmList = detector.getPosition(img)
#    plt.imshow(annotated_image)
#    plt.show()
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('projectYOGA-POSE.avi',cv2.VideoWriter_fourcc(*'DIVX'), 24, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

TypeError: create_bool(): incompatible function arguments. The following argument types are supported:
    1. (arg0: bool) -> mediapipe.python._framework_bindings.packet.Packet

Invoked with: 0.5

In [ ]:
# Initialize MediaPipe Holistic.
i=0
img_array = []
with mp_holistic.Holistic(
    static_image_mode=False,min_detection_confidence=0.8) as holistic:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Pose.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print nose coordinates.
    image_hight, image_width, _ = image.shape
    if results.pose_landmarks:
      print(
        f'Nose coordinates: ('
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_hight})'
      )
    # Draw pose landmarks.
    print(f'Pose landmarks of {name}:')
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.face_landmarks, 
        connections=mp_holistic.FACE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
    mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.pose_landmarks, 
        connections=mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
    #cv2_imshow(annotated_image)
    #cv2.imwrite("imag1", annotated_image)
    plt.figure(figsize=(10,15))
#    plt.imshow(annotated_image)
#    plt.show()
    height, width, layers = image.shape
    size = (width,height)
    img_array.append(annotated_image)
 
 
out = cv2.VideoWriter('projectYOGA.avi',cv2.VideoWriter_fourcc(*'DIVX'), 24, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
from abc import ABCMeta, abstractmethod

import numpy as np


class AbstDetector(metaclass=ABCMeta):
    @abstractmethod
    def detect(self, image: np.ndarray) -> bool:
        """モデルによる推論処理
        Args:
            image (np.ndarray): 入力イメージ
        Returns:
            bool: 対象物体（手や顔）が検出できたかどうか
        """
        pass

    @abstractmethod
    def draw(self, image: np.ndarray) -> np.ndarray:
        """推論結果を描画する
        Args:
            image (np.ndarray): ベースイメージ
        Returns:
            np.ndarray: 描画済みイメージ
        """
        pass

In [ ]:
import cv2
import numpy as np
import mediapipe as mp


from .abst_detector import AbstDetector


class PoseEstimator(AbstDetector):
    def __init__(self, min_detection_confidence: float, min_tracking_confidence: float) -> None:
        """初期化処理
        Args:
            min_detection_confidence (float): 姿勢推定モデルの最小信頼値
            min_tracking_confidence (float): ランドマーク追跡モデルからの最小信頼値
        """
        self.estimator = mp.solutions.pose.Pose(
            min_detection_confidence=min_detection_confidence,
            min_tracking_confidence=min_tracking_confidence,
        )

    def detect(self, image: np.ndarray) -> bool:
        """姿勢推定処理
        Args:
            image (np.ndarray): 入力イメージ
        Returns:
            bool: 手が検出できたか
        """
        try:
            self.results = self.estimator.process(image)
        except Exception as e:
            logger.error(e)
        return True if self.results.pose_landmarks is not None else False

    def draw(self, image: np.ndarray) -> np.ndarray:
        """処理結果を描画する
        Args:
            image (np.ndarray): ベースイメージ
        Returns:
            np.ndarray: 描画済みイメージ
        """
        landmark_buf = []
        base_width, base_height = image.shape[1], image.shape[0]

        # draw landmark points
        for landmark in self.results.pose_landmarks.landmark:
            x = min(int(landmark.x * base_width), base_width - 1)
            y = min(int(landmark.y * base_height), base_height - 1)
            landmark_buf.append((x, y))
            cv2.circle(image, (x, y), 2, (255, 0, 0), 5)

        # draw connections
        for con_pair in mp.solutions.pose.POSE_CONNECTIONS:
            cv2.line(image, landmark_buf[con_pair[0].value],
                     landmark_buf[con_pair[1].value], (255, 0, 0), 2)

        return image

In [ ]:
#face mesh 3d on camera

In [ ]:
!pip install mediapipe

In [ ]:
!pip install opencv-python

In [ ]:
Face Mesh 468 point 3D face landmarks in video even on mobile devices and web application MediaPipe

In [ ]:
#camera

In [ ]:
#image manipulation

In [ ]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt

#Import image
image = cv2.imread("image1073.jpg")
gray =cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#Show the image with matplotlib
plt.imshow(gray)
plt.show()

In [ ]:
import cv2
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline

# Acquire default dots per inch value of matplotlib
dpi = matplotlib.rcParams['figure.dpi']

img = cv2.imread("image1073.jpg")
img =cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# Determine the figures size in inches to fit your image
height, width, depth = img.shape
figsize = width / float(dpi), height / float(dpi)

plt.figure(figsize=figsize)
plt.imshow(img)
plt.show()

In [ ]:
figsize